In [2]:
import numpy as np
import pandas as pd 
from glob import glob
from os import path
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from torch.autograd import Variable
import torch.utils.data as utils
from time import sleep

In [31]:
N = 1200
batch_size = 32
seq_len = 20
num_features = 1
window_size = 10

In [32]:
def generate_data(N, seq_len):
    N_train = int(N*0.8)
    X = np.zeros((N, seq_len), dtype=np.float32)
    y = np.zeros((N,1), dtype=np.float32)
    
    indices = np.random.randint(0, N, size=N//2)
    
    X[indices, 0] = 1.0
    y[indices, 0] = 1.0
    
    X_train, y_train = X[:N_train], y[:N_train]
    X_test, y_test = X[N_train:], y[N_train:]
    
    return X_train, y_train, X_test, y_test
    

In [33]:
def generate_synthetic_test_data(N, seq_len):
    N_train = int(N*0.8)
    X = np.reshape(np.arange(0, N*seq_len, dtype=np.float32),(N, seq_len))
    y = np.reshape(np.arange(0,N, dtype=np.float32), (N,1))
    
    
    X_train, y_train = X[:N_train], y[:N_train]
    X_test, y_test = X[N_train:], y[N_train:]
    
    return X_train, y_train, X_test, y_test

In [34]:
X_train, y_train, X_test, y_test = generate_data(N, seq_len)

In [35]:
class StatefulSequenceDataset(object):
    def __init__(self, X, y, batch_size, seq_len, window_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.window_size = window_size
        N = X.shape[0]
        
        assert(N%batch_size == 0)
        assert(seq_len >= window_size)
        self.N = N
        
        # index into X,y matrices, ranges from 0:(N - batch_size)
        self.idx = 0
        
        # index into subsequence, ranges from 0:window_size
        self.subseq_idx = 0
        self.subseq_len = self.seq_len - self.window_size
        
    
    def get_batch(self):
        if self.subseq_idx == self.num_batches_per_sequence():
            self.idx = (self.idx + self.batch_size) % self.N
            self.subseq_idx = 0
        
        ii = self.idx
        bs = self.batch_size
        
        jj = self.subseq_idx
        xx = self.X[ii:ii+bs, jj:jj+self.window_size]        
        batch_X = torch.from_numpy(xx)
        batch_y = torch.from_numpy(self.y[ii:ii+bs])
        
        self.subseq_idx += 1
        return batch_X, batch_y
    
    def num_batches_per_epoch(self):
        return self.N//self.batch_size
    
    def num_batches_per_sequence(self):
        return (self.seq_len - self.window_size) + 1
    
    

In [44]:
class Net(nn.Module):
    def __init__(self, num_features, num_hidden, num_lstm_layers, batch_size):
        super(Net, self).__init__()        
        self.num_hidden = num_hidden
        self.num_lstm_layers = num_lstm_layers
        self.batch_size = batch_size
        self.lstm1 = nn.LSTM(input_size=1, hidden_size=num_hidden, 
                             num_layers=num_lstm_layers, batch_first=True)
        self.fc1 = nn.Linear(num_hidden, 1)
        self.sigmoid = nn.Sigmoid()
        self.hidden = None
        
    def init_hidden(self):
#         return (Variable(torch.zeros(self.num_lstm_layers, self.batch_size, self.num_hidden, dtype=torch.float32)), Variable(torch.zeros(self.num_lstm_layers, self.batch_size, self.num_hidden, dtype=torch.float32)))
        self.hidden = None
    
    def detach(self):
        for h in self.hidden:
            h = h.detach()
    
    def forward(self, x):
        if self.hidden is None:
            self.hidden = self.init_hidden()
        else:
            for h in self.hidden:
                h = Variable(h.data)
                
        lstm_out, self.hidden = self.lstm1(x, self.hidden)
        
        y_pred = self.sigmoid(self.fc1(lstm_out[:,-1]))
        return y_pred

In [48]:
ds = StatefulSequenceDataset(X_train, y_train, batch_size, seq_len, window_size)
num_epochs = 20
net = Net(num_features=1, num_hidden=64, num_lstm_layers=2, batch_size=batch_size)

In [49]:
learning_rate = 1e-3
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [50]:
hist = list()
for epoch in range(num_epochs):
    for _ in range(ds.num_batches_per_epoch()):
        net.init_hidden()
        loss = 0
        for _ in range(ds.num_batches_per_sequence()):
            optimizer.zero_grad()
            
            batch_data, batch_labels = ds.get_batch()
            batch_data = batch_data.unsqueeze(-1)
            y_pred = net(batch_data)
            loss += loss_fn(y_pred, batch_labels)

        loss.backward()
        optimizer.step()

    
    print("Epoch ", epoch, "BCE: ", loss.item())
            




Epoch  0 BCE:  7.997757911682129
Epoch  1 BCE:  7.88065242767334
Epoch  2 BCE:  7.914608955383301
Epoch  3 BCE:  7.902589321136475
Epoch  4 BCE:  7.907163619995117
Epoch  5 BCE:  7.914143085479736
Epoch  6 BCE:  0.15860700607299805
Epoch  7 BCE:  0.02989042177796364
Epoch  8 BCE:  0.01917487010359764
Epoch  9 BCE:  0.014581067487597466
Epoch  10 BCE:  0.011752137914299965
Epoch  11 BCE:  0.00976781826466322
Epoch  12 BCE:  0.008275449275970459
Epoch  13 BCE:  0.007115022279322147
Epoch  14 BCE:  0.006203509401530027
Epoch  15 BCE:  0.005482219625264406
Epoch  16 BCE:  0.0048978677950799465
Epoch  17 BCE:  0.004414767026901245
Epoch  18 BCE:  0.004009532742202282
Epoch  19 BCE:  0.003664357354864478


In [51]:
X_test_ = X_test[:224]
y_test_ = y_test[:224]

In [52]:
test_ds = StatefulSequenceDataset(X_test_, y_test_, batch_size=32,seq_len=seq_len, window_size=window_size)
               


In [53]:
xx, yy = test_ds.get_batch()

In [56]:
net.init_hidden()

In [57]:
net(xx.unsqueeze(-1))

tensor([[3.0509e-04],
        [9.9951e-01],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [9.9951e-01],
        [9.9951e-01],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [9.9951e-01],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [9.9951e-01],
        [9.9951e-01],
        [3.0509e-04],
        [9.9951e-01],
        [9.9951e-01],
        [9.9951e-01],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [3.0509e-04],
        [9.9951e-01],
        [9.9951e-01],
        [3.0509e-04]], grad_fn=<SigmoidBackward>)

In [55]:
yy

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.]])